In [9]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env
from gym_trading_env.wrapper import DiscreteActionsWrapper
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import wandb

# Vérification si on peut utiliser la carte graphique (GPU) ou juste le processeur (CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cerveau branché sur : {device}")

Cerveau branché sur : cuda


In [11]:
# --- CELLULE 2 OPTIMISÉE (Mode Turbo) ---

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.learning_rate = 0.001
        
        self.model = nn.Sequential(
            nn.Linear(state_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, action_size)
        ).to(device)
        
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        state = torch.FloatTensor(state).to(device)
        with torch.no_grad():
            act_values = self.model(state)
        return torch.argmax(act_values).item()

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        
        minibatch = random.sample(self.memory, batch_size)
        
        # --- OPTIMISATION VECTORIELLE (Le Turbo) ---
        # Au lieu d'une boucle for, on transforme tout le batch en gros tenseurs d'un coup
        
        # On empile les états (Batch, 1, 5) -> (Batch, 5)
        states = torch.FloatTensor(np.array([t[0] for t in minibatch])).squeeze(1).to(device)
        actions = torch.LongTensor(np.array([t[1] for t in minibatch])).unsqueeze(1).to(device)
        rewards = torch.FloatTensor(np.array([t[2] for t in minibatch])).to(device)
        next_states = torch.FloatTensor(np.array([t[3] for t in minibatch])).squeeze(1).to(device)
        dones = torch.FloatTensor(np.array([t[4] for t in minibatch])).to(device)

        # 1. Prédiction actuelle : Q(s, a)
        # On demande au réseau de calculer pour les 32 états d'un coup
        # .gather(1, actions) permet de ne garder que la valeur de l'action qui a été vraiment jouée
        current_q = self.model(states).gather(1, actions).squeeze(1)

        # 2. Prédiction future : max Q(s', a')
        # On calcule le max des actions futures possibles
        next_q = self.model(next_states).max(1)[0].detach()

        # 3. Calcul de la cible (Target)
        target_q = rewards + (self.gamma * next_q * (1 - dones))

        # 4. Apprentissage (Une seule passe pour tout le monde !)
        loss = self.criterion(current_q, target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [12]:
import glob
import os

def preprocess(df):
    """
    Fonction de prétraitement des données financières.
    Objectif : Rendre les données stationnaires pour le réseau de neurones.
    """
    # 1) Nettoyage structurel
    df = df.sort_index().dropna().drop_duplicates()

    # 2) Feature Engineering
    # Utilisation des rendements logarithmiques (log returns) préférables aux % bruts pour les réseaux de neurones
    # car ils sont symétriques et additifs.
    df["feature_close"] = np.log(df["close"]).diff()
    df["feature_high"] = np.log(df["high"]).diff()
    df["feature_low"] = np.log(df["low"]).diff()

    # on ignore le volume pour l'instant pour éviter les divisions par zéro sur le Forex/CFD
    # Si nécessaire plus tard, ajouter une gestion d'erreur spécifique (fillna ou epsilon).
    
    # 3) Nettoyage final (suppression des NaN générés par diff())
    df = df.dropna()
    
    return df

# Récupération de la liste des fichiers de données
# Assurez-vous que le dossier 'data' est à la racine du notebook
dataset_path = "./data/*.pkl" 
files = glob.glob(dataset_path)

if not files:
    print(f"WARNING: Aucun fichier trouvé dans {dataset_path}. Vérifiez le chemin.")
else:
    print(f"Data pipeline: {len(files)} fichiers détectés prêts pour l'environnement.")

Data pipeline: 9 fichiers détectés prêts pour l'environnement.


In [13]:
# Définition des métriques personnalisées pour le suivi
def metric_portfolio_valuation(history):
    return history['portfolio_valuation', -1]

# On définit le chemin sous forme de texte (String) avec une étoile pour dire "tous les pkl"
# C'est ce format que Windows et la librairie préfèrent.
dataset_path_str = "./data/*.pkl"

print(f"Chargement de l'environnement depuis : {dataset_path_str}")

# Création de l'environnement Gym
env_raw = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir=dataset_path_str,     # <--- CORRECTION ICI : On donne le string "*.pkl"
    preprocess=preprocess,            # Ta fonction définie dans la cellule 3
    portfolio_initial_value=1_000,    # Consigne: 1000$ initial
    trading_fees=0.1/100,             # Consigne: 0.1% frais
    borrow_interest_rate=0.02/100/24, # Consigne: 0.02% jour
    verbose=1
)

# Ajout de la métrique de valuation pour WandB
env_raw.add_metric('Valuation Finale', metric_portfolio_valuation)

# Wrapper pour actions discrètes
# Actions : 0 = Short (-1), 1 = Neutral (0), 2 = Long (1)
env = DiscreteActionsWrapper(env_raw, positions=[-1, 0, 1])

print(f"Environnement initialisé.")
print(f"Inputs (Ce que voit le robot) : {env.observation_space.shape}")
print(f"Outputs (Boutons disponibles) : {env.action_space.n}")

Chargement de l'environnement depuis : ./data/*.pkl
Environnement initialisé.
Inputs (Ce que voit le robot) : (5,)
Outputs (Boutons disponibles) : 3


In [15]:
# On initialise une nouvelle expérience.
# "id" permet de reprendre une courbe si ça plante, mais ici on laisse vide pour une nouvelle.
wandb.init(
    project="Projet-Trading-RL",
    name="DQN-Baseline-Local-GPU",
    config={
        "architecture": "Vanilla DQN",
        "dataset": "Multi-Asset (Binance & Yahoo)",
        "features": "Log Returns",
        "initial_capital": 1000,
        "gamma": 0.95,
        "epsilon_start": 1.0,
        "epsilon_decay": 0.995,
        "learning_rate": 0.001,
        "batch_size": 32,
        "episodes": 20  # Tu pourras augmenter ce chiffre plus tard
    }
)

print(" WandB connecté. Les courbes vont apparaître en ligne.")

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


 WandB connecté. Les courbes vont apparaître en ligne.


In [16]:

# Paramètres de la session
EPISODES = 20           # Nombre de parties à jouer
BATCH_SIZE = 32         # Taille du lot pour l'apprentissage

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

print(f"Démarrage de l'entraînement sur {device} pour {EPISODES} épisodes...")
print("-" * 50)

try:
    for e in range(1, EPISODES + 1):
        state, info = env.reset()
        
        # Formatage state
        if isinstance(state, tuple): state = state[0]
        state = np.reshape(state, [1, state_size])
        
        done = False
        total_reward = 0
        step_count = 0
        
        while not done:
            # Action
            action = agent.act(state)
            
            # Step
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            # Formatage next_state
            next_state = np.reshape(next_state, [1, state_size])
            
            # Mémorisation
            agent.remember(state, action, reward, next_state, done)
            
            state = next_state
            total_reward += reward
            step_count += 1
            
            # Replay
            if len(agent.memory) > BATCH_SIZE:
                agent.replay(BATCH_SIZE)
            
            if step_count % 500 == 0:
                print(f"   Episode {e} | Step {step_count} | Val: {info['portfolio_valuation']:.0f}$ | Eps: {agent.epsilon:.2f}", end='\r')

        # --- Fin de l'épisode ---
        
        # on utilise .unwrapped pour accéder aux métriques
        metrics = env.unwrapped.get_metrics()
        final_val = metrics['Valuation Finale']
        
        print(f"Episode {e}/{EPISODES} Terminé.")
        # On nettoie les % pour l'affichage (s'ils sont en string)
        p_return = metrics['Portfolio Return']
        print(f"   Score: {final_val:.2f}$ | ROI: {p_return} | Epsilon: {agent.epsilon:.3f}")
        print("-" * 50)
        
        # Envoi à WandB
        # Petite sécurité supplémentaire pour convertir les strings "10%" en float 10.0
        try:
            market_ret = float(str(metrics['Market Return']).strip('%'))
        except: market_ret = 0.0
        
        try:
            port_ret = float(str(metrics['Portfolio Return']).strip('%'))
        except: port_ret = 0.0

        wandb.log({
            "Episode": e,
            "Valuation": final_val,
            "Epsilon": agent.epsilon,
            "Market Return": market_ret,
            "Portfolio Return": port_ret
        })

except KeyboardInterrupt:
    print("\nEntraînement interrompu manuellement.")

finally:
    wandb.finish()
    env.close()
    print("Session terminée.")

Démarrage de l'entraînement sur cuda pour 20 épisodes...
--------------------------------------------------
Market Return : 902.89%   |   Portfolio Return : -100.00%   |   Valuation Finale : 0.020621328804258432   |   
Episode 1/20 Terminé.
   Score: 0.02$ | ROI: -100.00% | Epsilon: 0.010
--------------------------------------------------
Market Return : 40.24%   |   Portfolio Return : -56.48%   |   Valuation Finale : 435.18757484630504   |   
Episode 2/20 Terminé.
   Score: 435.19$ | ROI: -56.48% | Epsilon: 0.010
--------------------------------------------------
Market Return : 27.57%   |   Portfolio Return : -56.95%   |   Valuation Finale : 430.5303087160685   |   
Episode 3/20 Terminé.
   Score: 430.53$ | ROI: -56.95% | Epsilon: 0.010
--------------------------------------------------
Market Return : 209.83%   |   Portfolio Return : -54.73%   |   Valuation Finale : 452.731540647851   |   
Episode 4/20 Terminé.
   Score: 452.73$ | ROI: -54.73% | Epsilon: 0.010
----------------------

Episode,▁▂▃▅▆▇█
Epsilon,▁▁▁▁▁▁▁
Market Return,█▁▁▃▁▂▁
Portfolio Return,▁████▂▁
Valuation,▁████▂▁
Episode,7
Epsilon,0.01
Market Return,5.19
Portfolio Return,-97.23
Valuation,27.65345


Session terminée.
